In [1]:
import os
import sys

def set_root_path():
    if os.getcwd().endswith('figures'): 
        os.chdir('../')
set_root_path()
sys.path.append('python/')
sys.path = [p for p in sys.path if "unbiased-inverse-volume-rendering" not in p]
print(os.getcwd())
print(sys.executable)
os.environ.pop("PYTHONPATH")

c:\Users\mayuf\source\repos\practical-inverse-rendering-of-textured-and-translucent-appearance
c:\Users\mayuf\anaconda3\envs\practical-inverse-rendering\python.exe


'C:\\Users\\mayuf\\source\\repos\\unbiased-inverse-volume-rendering\\mitsuba3\\build\\Release\\python'

In [2]:
from pathlib import Path
import mitsuba as mi
import numpy as np
mi.set_variant('cuda_ad_rgb')

from practical_reconstruction import optimization_cli
from core import integrators
from core import bsdfs
from core import textures

integrators.register()
bsdfs.register()
textures.register()

In [3]:
def _update_light_position(params, camera_idx):
	light_positions = [
["4.554923 0.152407 -0.205656 10.582809 0.000000 4.986326 0.036953 4.152369 2.062201 -0.336632 0.454247 -23.212334 0.000000 0.000000 0.000000 1.000000"],
["-4.554923 0.846602 0.188041 -9.102048 0.000000 4.559228 -0.205267 16.263325 -2.062202 -1.869947 -0.415339 20.266935 0.000000 0.000000 0.000000 1.000000"],
["3.982244 1.183225 0.278239 -13.611968 0.000000 4.601233 -0.195670 15.783475 -3.023530 1.558407 0.366465 -18.823231 0.000000 0.000000 0.000000 1.000000"],
["-3.982244 0.160232 -0.301928 15.396410 -0.000000 4.992974 0.026498 4.675113 3.023530 0.211040 -0.397665 19.383238 0.000000 0.000000 0.000000 1.000000"],
["4.137138 -1.925474 -0.204368 10.518407 -0.000000 3.639213 -0.342872 23.143600 2.807861 2.837019 0.301119 -15.555934 0.000000 0.000000 0.000000 1.000000"],
["-4.137139 -1.110372 0.257898 -12.594914 0.000000 4.592435 0.197726 -3.886291 -2.807860 1.636037 -0.379991 18.499538 0.000000 0.000000 0.000000 1.000000"],
["4.354821 2.076192 0.131338 -6.266884 -0.000000 2.673016 -0.422552 27.127579 -2.456732 3.680273 0.232810 -12.140509 0.000000 0.000000 0.000000 1.000000"],
["-4.354821 1.501780 -0.194427 10.021350 -0.000001 3.957026 0.305646 -9.282302 2.456732 2.662068 -0.344643 16.732138 0.000000 0.000000 0.000000 1.000000"],
["3.334635 -0.561823 -0.368301 18.715048 -0.000000 4.942822 -0.075400 9.770000 3.725615 0.502863 0.329650 -16.982510 0.000000 0.000000 0.000000 1.000000"],
["-3.334636 0.731723 0.365305 -17.965258 -0.000000 4.902616 -0.098202 10.910082 -3.725615 -0.654933 -0.326969 15.848438 0.000000 0.000000 0.000000 1.000000"],
["3.318174 -0.080402 0.373942 -18.397095 -0.000000 4.998845 0.010748 5.462587 -3.740283 -0.071329 0.331741 -17.087040 0.000000 0.000000 0.000000 1.000000"],
["-3.318176 -1.354511 -0.348640 17.732023 0.000000 4.660616 -0.181071 15.053531 3.740282 -1.201649 -0.309295 14.964740 0.000000 0.000000 0.000000 1.000000"],
["2.751413 2.757752 0.313441 -15.372048 0.000000 3.753881 -0.330278 22.513916 -4.174892 1.817464 0.206570 -10.828478 0.000000 0.000000 0.000000 1.000000"],
["-2.751414 1.519408 -0.388859 19.742943 -0.000000 4.657113 0.181970 -3.098490 4.174892 1.001348 -0.256273 12.313642 0.000000 0.000000 0.000000 1.000000"],
["2.473057 -2.507945 -0.354883 18.044151 0.000000 4.083273 -0.288563 20.428169 4.345572 1.427267 0.201963 -10.598166 0.000000 0.000000 0.000000 1.000000"],
["-2.473057 -1.142926 0.419258 -20.662889 0.000000 4.823967 0.131505 -0.575236 -4.345572 0.650437 -0.238599 11.429943 0.000000 0.000000 0.000000 1.000000"],
["3.503841 -1.832380 0.306031 -15.001532 0.000000 4.289815 0.256856 -6.842788 -3.566945 -1.799963 0.300617 -15.530832 0.000000 0.000000 0.000000 1.000000"],
["-3.503842 -2.768561 -0.224904 11.545181 -0.000000 3.152609 -0.388086 25.404289 3.566944 -2.719583 -0.220925 10.546243 0.000000 0.000000 0.000000 1.000000"],
["2.194997 0.955449 -0.438966 22.248293 -0.000000 4.885610 0.106340 0.683013 4.492436 -0.466831 0.214478 -11.223903 0.000000 0.000000 0.000000 1.000000"],
["-2.194998 2.399179 0.379815 -18.690737 -0.000000 4.227269 -0.267024 19.351219 -4.492436 -1.172236 -0.185577 8.778844 0.000000 0.000000 0.000000 1.000000"],
["1.691467 1.062233 0.458373 -22.618662 0.000000 4.870918 -0.112879 11.643924 -4.705204 0.381861 0.164780 -8.738997 0.000000 0.000000 0.000000 1.000000"],
["-1.691467 -0.569557 -0.467060 23.653021 0.000001 4.963233 -0.060524 9.026206 4.705204 -0.204750 -0.167903 7.895144 0.000000 0.000000 0.000000 1.000000"],
["2.195768 -4.129715 -0.176750 9.137499 0.000000 1.967359 -0.459668 28.983416 4.492060 2.018650 0.086397 -4.819869 0.000000 0.000000 0.000000 1.000000"],
["-2.195770 -3.276142 0.307335 -15.066760 -0.000000 3.420872 0.364659 -12.232962 -4.492059 1.601415 -0.150229 7.011445 0.000000 0.000000 0.000000 1.000000"],
["1.586890 4.247731 0.210679 -10.233948 -0.000000 2.221651 -0.447932 28.396576 -4.741496 1.421636 0.070510 -4.025518 0.000000 0.000000 0.000000 1.000000"],
["-1.586891 3.270996 -0.343254 17.462721 0.000000 3.619686 0.344933 -11.246651 4.741495 1.094742 -0.114881 5.244046 0.000000 0.000000 0.000000 1.000000"],
["0.973068 -1.466863 -0.467990 23.699492 0.000000 4.771122 -0.149546 13.477283 4.904400 0.291036 0.092853 -5.142629 0.000000 0.000000 0.000000 1.000000"],
["-0.973069 0.222220 0.489936 -24.196817 -0.000001 4.994865 -0.022655 7.132752 -4.904400 -0.044091 -0.097207 4.360346 0.000000 0.000000 0.000000 1.000000"],
["1.026869 -1.612397 0.462014 -22.800713 0.000000 4.720772 0.164752 -2.237581 -4.893418 -0.338357 0.096952 -5.347619 0.000000 0.000000 0.000000 1.000000"],
["-1.026870 -3.095339 -0.379004 19.250206 0.000000 3.872591 -0.316276 21.813789 4.893418 -0.649549 -0.079533 3.476647 0.000000 0.000000 0.000000 1.000000"],
["0.767473 2.823286 -0.405463 20.573160 0.000000 4.103258 0.285714 -8.285723 4.940747 -0.438556 0.062983 -3.649141 0.000000 0.000000 0.000000 1.000000"],
["-0.767474 4.039787 0.284449 -13.922436 -0.000000 2.878599 -0.408823 26.441174 -4.940747 -0.627523 -0.044185 1.709249 0.000000 0.000000 0.000000 1.000000"],
["0.271717 2.694309 0.420320 -20.715988 0.000000 4.209418 -0.269830 19.491480 -4.992611 0.146635 0.022875 -1.643767 0.000000 0.000000 0.000000 1.000000"],
["-0.271717 1.094244 -0.487122 24.656107 0.000000 4.878430 0.109586 0.520683 4.992611 0.059553 -0.026511 0.825549 0.000000 0.000000 0.000000 1.000000"],
["0.101845 -3.818668 -0.322605 16.430250 0.000000 3.226719 -0.381946 25.097305 4.998962 0.077799 0.006573 -0.828625 0.000000 0.000000 0.000000 1.000000"],
["-0.101846 -2.485000 0.433756 -21.387785 0.000001 4.338457 0.248552 -6.427580 -4.998962 0.050628 -0.008837 -0.058150 0.000000 0.000000 0.000000 1.000000"],
["-0.478657 0.227147 -0.497185 25.159248 -0.000000 4.994790 0.022820 4.859016 4.977036 0.021846 -0.047816 1.890789 0.000000 0.000000 0.000000 1.000000"],
["0.478656 1.913923 0.459432 -22.671608 -0.000000 4.615520 -0.192275 15.613768 -4.977036 0.184067 0.044185 -2.709249 0.000000 0.000000 0.000000 1.000000"],
["-0.656736 0.032497 0.495658 -24.482880 -0.000001 4.999892 -0.003278 6.163902 -4.956682 -0.004306 -0.065672 2.783611 0.000000 0.000000 0.000000 1.000000"],
["0.656736 -1.664713 -0.466877 23.643858 -0.000000 4.709573 -0.167926 14.396303 4.956682 0.220566 0.061859 -3.592948 0.000000 0.000000 0.000000 1.000000"],
["-1.351134 3.781045 0.297962 -14.598101 0.000000 3.094755 -0.392715 25.635742 -4.813983 -1.061220 -0.083629 3.681424 0.000000 0.000000 0.000000 1.000000"],
["1.351134 2.533931 -0.409312 20.765602 -0.000000 4.251282 0.263184 -7.159217 4.813983 -0.711195 0.114881 -6.244046 0.000000 0.000000 0.000000 1.000000"],
["-1.343589 -2.364195 -0.419587 21.279366 -0.000000 4.356095 -0.245447 18.272366 4.816095 -0.659560 -0.117056 5.352798 0.000000 0.000000 0.000000 1.000000"],
["1.343587 -0.786542 0.475143 -23.457169 -0.000000 4.932870 0.081658 1.917110 -4.816095 -0.219429 0.132555 -7.127746 0.000000 0.000000 0.000000 1.000000"],
["-2.287396 -2.151856 0.389067 -19.153366 0.000001 4.375375 0.241994 -6.099680 -4.446102 1.107071 -0.200164 9.508217 0.000000 0.000000 0.000000 1.000000"],
["2.287395 -3.352772 -0.292006 14.900292 0.000000 3.283841 -0.377046 24.852312 4.446102 1.724908 0.150229 -8.011445 0.000000 0.000000 0.000000 1.000000"],
["-2.391057 1.436341 -0.414967 21.048359 -0.000000 4.724960 0.163547 -2.177340 4.391223 0.782100 -0.225953 10.797644 0.000000 0.000000 0.000000 1.000000"],
["2.391056 2.768991 0.340816 -16.740793 -0.000000 3.880646 -0.315287 21.764343 -4.391224 1.507737 0.185577 -9.778844 0.000000 0.000000 0.000000 1.000000"],
["-2.032528 1.581548 0.428574 -21.128681 -0.000000 4.690794 -0.173103 14.655125 -4.568241 -0.703672 -0.190683 9.034175 0.000000 0.000000 0.000000 1.000000"],
["2.032528 0.020361 -0.456820 23.140980 -0.000000 4.999950 0.002228 5.888574 4.568242 -0.009058 0.203251 -10.662543 0.000000 0.000000 0.000000 1.000000"],
["-2.768702 -3.494389 -0.226352 11.617624 -0.000001 2.718329 -0.419651 26.982542 4.163447 -2.323776 -0.150525 7.026241 0.000000 0.000000 0.000000 1.000000"],
["2.768700 -2.509480 0.332217 -16.310846 0.000000 3.989685 0.301370 -9.068522 -4.163448 -1.668808 0.220925 -11.546243 0.000000 0.000000 0.000000 1.000000"],
["-3.512960 3.174526 0.160670 -7.733514 0.000001 2.257896 -0.446115 28.305777 -3.557964 -3.134372 -0.158638 7.431901 0.000000 0.000000 0.000000 1.000000"],
["3.512961 2.433554 -0.259556 13.277793 -0.000000 3.647534 0.341987 -11.099342 3.557964 -2.402772 0.256273 -13.313642 0.000000 0.000000 0.000000 1.000000"],
["-2.770702 -0.821877 -0.408016 20.700815 -0.000000 4.901548 -0.098733 10.936649 4.162116 -0.547119 -0.271615 13.080730 0.000000 0.000000 0.000000 1.000000"],
["2.770702 0.623186 0.411520 -20.275990 -0.000001 4.943636 -0.074864 9.743209 -4.162116 0.414851 0.273947 -14.197341 0.000000 0.000000 0.000000 1.000000"],
["-3.234591 -0.374375 0.379437 -18.671864 0.000000 4.975839 0.049095 3.545277 -3.812797 0.317601 -0.321896 15.594807 0.000000 0.000000 0.000000 1.000000"],
["3.234591 -1.649549 -0.343750 17.487501 -0.000000 4.507846 -0.216317 16.815870 3.812797 1.399397 0.291621 -15.081041 0.000000 0.000000 0.000000 1.000000"],
["-3.561214 1.992272 0.288940 -14.146979 -0.000000 4.116340 -0.283826 20.191320 -3.509666 -2.021534 -0.293183 14.159169 0.000000 0.000000 0.000000 1.000000"],
["3.561214 0.883892 -0.339654 17.282705 0.000000 4.838838 0.125923 -0.296121 3.509665 -0.896875 0.344643 -17.732138 0.000000 0.000000 0.000000 1.000000"],
["-3.752037 -1.898442 -0.270521 13.826030 -0.000001 4.092747 -0.287218 20.360905 3.304878 -2.155306 -0.307123 14.856137 0.000000 0.000000 0.000000 1.000000"],
["3.752037 -0.858717 0.319137 -15.656837 0.000000 4.828268 0.129917 -0.495827 -3.304878 -0.974903 0.362317 -18.615837 0.000000 0.000000 0.000000 1.000000"],
["-4.328433 -0.159333 -0.249785 12.789277 -0.000000 4.989859 -0.031829 7.591456 2.502932 -0.275540 -0.431965 21.098270 0.000000 0.000000 0.000000 1.000000"],
["4.328433 0.704594 0.240171 -11.708556 0.000000 4.797795 -0.140754 13.037682 -2.502933 1.218486 0.415339 -21.266935 0.000000 0.000000 0.000000 1.000000"],
["-4.354185 0.593537 0.238512 -11.625589 0.000000 4.852023 -0.120743 12.037126 -2.457859 -1.051471 -0.422532 20.626606 0.000000 0.000000 0.000000 1.000000"],
["4.354185 -0.258017 -0.244428 12.521396 0.000000 4.972373 -0.052488 8.624405 2.457860 0.457086 0.433013 -22.150637 0.000000 0.000000 0.000000 1.000000"],
	]
	raw_string = light_positions[camera_idx][0]
	float_values = list(map(float, raw_string.strip().split()))
	matrix_4x4 = np.array(float_values).reshape((4, 4))
	params['arealight.to_world'] = mi.Transform4f(matrix_4x4)
	params.update()
	# print("after(no T):\n", np.array(params['arealight.to_world']))

def save_images(scene, params, dir, spp):
    print("start rendering")
    for i, sensor in enumerate(scene.sensors()):
					_update_light_position(params, i)
											
					integrator = mi.load_dict({
							'type': 'prb_path_volume',
							'max_sss_depth': 256,
							'max_path_depth': 3,
					})
					image = mi.render(scene, sensor=sensor, integrator=integrator, spp=spp)
					bitmap = mi.Bitmap(image).convert(
									pixel_format=mi.Bitmap.PixelFormat.RGB,
									component_format=mi.Struct.Type.UInt8,
									srgb_gamma=True
					)
					png_path = dir / f"ref_view_{i:03d}.png"
					bitmap.write(str(png_path))

					bitmap = mi.Bitmap(image).convert(
													pixel_format=mi.Bitmap.PixelFormat.RGB,
													component_format=mi.Struct.Type.Float32,
													srgb_gamma=False
					)
					exr_path = dir / f"ref_view_{i:03d}.exr"
					bitmap.write(str(exr_path))
					print(f"saving: {i}")

In [ ]:
ref_spp=4096
scene_name = 'kiwi'
ref_dir=Path("third_party/kiwi/references")
ref_scene_path=f'third_party/{scene_name}/mts_scene/kiwi_ref.xml'
ref_scene = mi.load_file(ref_scene_path)
ref_params = mi.traverse(ref_scene)

save_images(ref_scene, ref_params, ref_dir, ref_spp)

start rendering
saving: 0
saving: 1
saving: 2
saving: 3
saving: 4
saving: 5
saving: 6
saving: 7
saving: 8
saving: 9
saving: 10
saving: 11
saving: 12
saving: 13
saving: 14
saving: 15
saving: 16
saving: 17
saving: 18
saving: 19
saving: 20
saving: 21
saving: 22
saving: 23
saving: 24
saving: 25
saving: 26
saving: 27
saving: 28
saving: 29
saving: 30
saving: 31
saving: 32
saving: 33
saving: 34
saving: 35
saving: 36
saving: 37
saving: 38
saving: 39
saving: 40
saving: 41
saving: 42
saving: 43
saving: 44
saving: 45
saving: 46
saving: 47
saving: 48
saving: 49
saving: 50
saving: 51
saving: 52
saving: 53
saving: 54
saving: 55
saving: 56
saving: 57
saving: 58
saving: 59
saving: 60
saving: 61
saving: 62
saving: 63
saving: 64
saving: 65


In [ ]:
def format_float(f):
  """Formats a float such that 0.1 becomes "0_1", 10.0 becomes "10_0", etc."""
  return str(f).replace('.', '_')

scene_name = 'kiwi'
technique = 'mipmap_pyramid'
base_learning_rate = 0.0005

skip_existing = False

print(
    f'******** Running {technique} with base learning rate'
    f' {base_learning_rate} ********'
)

override_bindings = []
result_folder = f'results/{scene_name}/{technique}'

result_folder += f'_lr_{format_float(base_learning_rate)}'
override_bindings.append(
    f'SceneConfig.base_learning_rate={base_learning_rate}'
)

# Ensure that the default tmp folder is used
override_bindings.append("SceneConfig.tmp_folder=''")
override_bindings.append(
    f'SceneConfig.use_gradient_filtering=False'
)

override_bindings.append(
    f"SceneConfig.result_folder='{result_folder}'"
)

if technique == 'gradient_filtering':
    gin_config_name = f'{scene_name}/naive'
else:
    gin_config_name = f'{scene_name}/{technique}'

print(f'Next result location: {result_folder}')
if skip_existing and Path(result_folder).exists():
    print('Skipping, already present')
else:
    # Run the config
    optimization_cli.run_config(gin_config_name, override_bindings, sss_config=True)

******** Running mipmap_pyramid with base learning rate 0.0005 ********
Next result location: results/kiwi/mipmap_pyramid_lr_0_0005
Preparing Mitsuba scene for optimization
Preparing references and sensors for optimization
Rendering camera elm__1
Reference found at: third_party\kiwi\references\ref_view_000.exr, copying to tmp folder
Rendering camera elm__2
Reference found at: third_party\kiwi\references\ref_view_001.exr, copying to tmp folder
Rendering camera elm__3
Reference found at: third_party\kiwi\references\ref_view_002.exr, copying to tmp folder
Rendering camera elm__4
Reference found at: third_party\kiwi\references\ref_view_003.exr, copying to tmp folder
Rendering camera elm__5
Reference found at: third_party\kiwi\references\ref_view_004.exr, copying to tmp folder
Rendering camera elm__6
Reference found at: third_party\kiwi\references\ref_view_005.exr, copying to tmp folder
Rendering camera elm__7
Reference found at: third_party\kiwi\references\ref_view_006.exr, copying to tmp 

Loading references for sensor elm__66: 100%|██████████| 1/1 [00:00<00:00, 400.22it/s]


Preparing optimization variables
Optimizing mat-kiwi's single_scattering_albedo from default value : [[0.5, 0.5, 0.5]]
Optimizing mat-kiwi's extinction_coefficient from default value : [[0.01, 0.01, 0.01]]
Optimizing mat-kiwi's hg_coefficient from default value : [[0.1, 0.1, 0.1]]
Learning rate for (texture) mat-kiwi (single_scattering_albedo) is  0.0005
Learning rate for (scalar) mat-kiwi (extinction_coefficient) is  0.0005
Learning rate for (texture) mat-kiwi (hg_coefficient) is  0.0005
Starting optimization
Resizing initial value for mat-kiwi.nested_bsdf.single_scattering_albedo.data/mat-kiwi.nested_bsdf.single_scattering_albedo.data to shape: (128, 128, 3)
Skipping resizing as target shape is already reached.
Resizing initial value for mat-kiwi.nested_bsdf.hg_coefficient.data/mat-kiwi.nested_bsdf.hg_coefficient.data to shape: (512, 512, 3)
Skipping resizing as target shape is already reached.


Rendering at resolution level 0 (width=436):   0%|          | 0/4096 [00:00<?, ?it/s]

0.0228457
0.019347
0.0697317


Rendering at resolution level 0 (width=436):   0%|          | 1/4096 [00:04<5:37:40,  4.95s/it]

0.264431


Rendering at resolution level 0 (width=436):   1%|▏         | 56/4096 [03:33<3:28:18,  3.09s/it] 

# Figure starts here

In [ ]:
import matplotlib.pyplot as plt
from practical_reconstruction import figutils
from matplotlib import gridspec

def figure_grid_setup(image_shape,image_crop_shape,n_columns = 7,inner_space=0.0,outer_space=0.1,figwidth=figutils.TEXT_WIDTH):
  # Image aspect ratios
  h, w = image_shape
  h_crop, w_crop = image_crop_shape

  top_inner_rows = 1
  top_inner_cols = n_columns
  # Spacing in the inner gridspec
  top_inner_wspace = inner_space
  # same vertical spacing as horizontal spacing
  top_inner_hspace = top_inner_wspace * figutils.gridspec_aspect(
      n_rows=1, n_cols=1, w=[w]*top_inner_cols, h=[h]
  )
  top_height_ratios = [h]
  top_inner_aspect = figutils.gridspec_aspect(
      n_rows=top_inner_rows,
      n_cols=top_inner_cols,
      w=[w] * top_inner_cols,
      h=top_height_ratios,
      wspace=top_inner_wspace,
      hspace=top_inner_hspace,
  )
  # print(top_inner_aspect)

  # Spacing in the main griddpec
  outer_rows = 3
  outer_cols = 1
  outer_wspace = 0.0
  outer_hspace = outer_space
  # If width is 1, we need the sum of the inverses for the height (single column)
  # If height is 1, we need the sum for the width (single row)
  outer_aspect = figutils.gridspec_aspect(
      n_rows=outer_rows,
      n_cols=outer_cols,
      w=1,
      h=[1 / top_inner_aspect] * outer_rows,
      wspace=outer_wspace,
      hspace=outer_hspace,
  )

  fig = plt.figure(
      1, figsize=(figwidth, figwidth / outer_aspect)
  )

  outer_gs = fig.add_gridspec(
      outer_rows,
      outer_cols,
      hspace=outer_hspace,
      wspace=outer_wspace,
      height_ratios=[1 / top_inner_aspect] * outer_rows,
      width_ratios=[1]*outer_cols,
  )

  top_inner_gss = []
  for row in range(outer_rows):
    top_inner_gs = gridspec.GridSpecFromSubplotSpec(
        top_inner_rows,
        top_inner_cols,
        subplot_spec=outer_gs[row],
        wspace=top_inner_wspace,
        hspace=top_inner_wspace,
        width_ratios=[h] * top_inner_cols,
        height_ratios=top_height_ratios,
    )
    top_inner_gss.append(top_inner_gs)

  return (
      fig,
      (top_inner_gss, top_inner_rows, top_inner_cols),
  )

In [ ]:
import drjit as dr
import numpy as np
from core import mitsuba_io
from core import image_util

def l2_error(ref, img):
  return dr.mean(dr.square(ref - img)).array[0]


def l1_error(ref, img):
  return dr.mean(dr.abs(ref - img)).array[0]

sensor_indices = [20, 1, 25, 12]

lr = '0_0005'
normalmap_lr = '0_01'
scene_folder = 'results/kiwi'

ref_scene_folder = (
    'third_party/kiwi/unmasked_references'
)
ref_masked_scene_folder = (
    'third_party/kiwi/references'
)
deng_scene_folder = (
    'third_party/kiwi/deng_optimized/'
)

crop_sizes = [
    (110, 110),
    (126, 126),
    (94, 94),
    (113, 113),
]

crop_offsets = [
    (159, 92),
    (150, 75),
    (168, 96),
    (157, 86),
]

our_images = []
ref_masked_images = []
ref_images = []
deng_images = []
errors_l2_ours = []
errors_l2_deng = []

boost = np.sqrt(2)

for sensor_idx, crop_size, crop_offset in zip(sensor_indices,crop_sizes,crop_offsets):

  our_filename = f'{scene_folder}/mipmap_pyramid_lr_{lr}/frames/camera_{sensor_idx:03d}_iter_4095_spp_8192.exr'
  our_image = mitsuba_io.read_bitmap(our_filename).convert(
      pixel_format=mi.Bitmap.PixelFormat.RGB,
      component_format=mi.Struct.Type.Float32,
  )
  our_images.append(image_util.tonemap(boost*our_image))
  ref_masked_filename = (
      f'{ref_masked_scene_folder}/ref_view_{sensor_idx:03d}.exr'
  )
  ref_masked_image = mitsuba_io.read_bitmap(ref_masked_filename).convert(
      pixel_format=mi.Bitmap.PixelFormat.RGB,
      component_format=mi.Struct.Type.Float32,
  )
  ref_masked_images.append(image_util.tonemap(ref_masked_image))

  ref_filename = f'{ref_scene_folder}/kiwi_{sensor_idx:05d}.exr'
  ref_image = mitsuba_io.read_bitmap(ref_filename).convert(
      pixel_format=mi.Bitmap.PixelFormat.RGB,
      component_format=mi.Struct.Type.Float32,
  )
  ref_images.append(
      image_util.tonemap(
          boost*image_util.resize_to_width(ref_image, ref_image.size()[0] // 2)
      )
  )
  deng_filename = f'{deng_scene_folder}/optimized_{sensor_idx:03d}_spp_256.exr'
  deng_image = mitsuba_io.read_bitmap(deng_filename).convert(
      pixel_format=mi.Bitmap.PixelFormat.RGB,
      component_format=mi.Struct.Type.Float32,
  )
  deng_images.append(image_util.tonemap(boost*deng_image))

  errors_l2_ours.append(
      l2_error(
          figutils.crop_image(
              mi.TensorXf(ref_masked_image), crop_offset, crop_size
          ),
          figutils.crop_image(
              mi.TensorXf(our_image), crop_offset, crop_size
          ),
      )
  )
  errors_l2_deng.append(
      l2_error(
          figutils.crop_image(
              mi.TensorXf(ref_masked_image), crop_offset, crop_size
          ),
          figutils.crop_image(
              mi.TensorXf(deng_image), crop_offset, crop_size
          ),
      )
  )

our_images = [
    figutils.crop_image(np.array(image), crop_offsets[i], crop_sizes[i])
    for i,image in enumerate(our_images)
]
ref_images = [
    figutils.crop_image(np.array(image), crop_offsets[i], crop_sizes[i])
    for i,image in enumerate(ref_images)
]
deng_images = [
    figutils.crop_image(np.array(image), crop_offsets[i], crop_sizes[i])
    for i,image in enumerate(deng_images)
]

In [ ]:
FIGURE_DIR = "figures/pdfs"
FIGURE_NAME = "deng_comparison"

def error_format(error, scale):
  return f"{error*scale:.3f}"

(
    fig,
    (top_inner_gss, top_inner_rows, top_inner_cols),
) = figure_grid_setup(
    our_images[0].shape[:2],
    our_images[0].shape[:2],
    n_columns=len(sensor_indices),
    inner_space=0.03,
    outer_space=0.15,
    figwidth=figutils.COLUMN_WIDTH,
)


scale = 1000
scale_txt = figutils.math_label(r"\text{$\times 10^3$}")

line_width = 0.75
crop_color = 'orange'
crop_color = (55/255.0,118/255.0,171/255.0,1.0)

row_titles = [
    "Deng et al. 2022",
    "Ours",
    "Reference",
]

for row, top_inner_gs in enumerate(top_inner_gss):
  if row == 0:
    images = deng_images
    errors = errors_l2_deng
  elif row == 1:
    images = our_images
    errors = errors_l2_ours
  elif row == 2:
    images = ref_images

  for col in range(top_inner_cols):
    ax = fig.add_subplot(top_inner_gs[col])
    ax.spines[:].set_color(crop_color)
    ax.spines[:].set_linewidth(line_width)
    figutils.disable_ticks(ax)
    image = images[col]

    # alpha_channel = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8) + 255
    # black_pixels = np.all(image == 0, axis=-1)
    # alpha_channel[black_pixels] = 0
    # image_with_alpha = np.dstack((image, alpha_channel))

    ax.imshow(image, aspect="equal")
    if col == 0:
      ax.set_ylabel(row_titles[row],labelpad=1.5)

    if row == 0 or row == 1:
      label = error_format(errors[col], scale)
    elif row == 2:
      label = r"$L_2$ " + scale_txt + " error" if col == 0 else ""
    ax.set_xlabel(label, labelpad=1.5)

figutils.force_post_crop_size(fig, figutils.COLUMN_WIDTH)

In [ ]:
figutils.savefig(
    fig,
    name=Path(FIGURE_NAME),
    fig_directory=Path(FIGURE_DIR),
    dpi=300,
    pad_inches=0.005,
    bbox_inches="tight",
    compress=False,
    target_width=figutils.COLUMN_WIDTH,
    backend=None,
)